# OTROS TIEMPOS

In [97]:
weeks =2

In [98]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

lost_sales_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "search",
    [[["inicio", ">=", FECHA_INICIO], ["inicio", "<=", FECHA_FIN]]]
)

lost_sales_data = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.operarios.tiempos", "read",
    [lost_sales_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df = pd.DataFrame(lost_sales_data)

df['tarea'] = df['tarea_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

#Tarda  menos de 1 minuto

📅 Descargando registros desde 2025-10-13 hasta 2025-10-27


In [99]:
df.head()

,create_uid,__last_update,wk_line_id,create_date,name,tipo,tarea_id,delay_pausa,is_pause,finalizo,delay_parcial,delay_total,production_id,inicio,write_date,workcenter_id,costo_total,write_uid,id,delay_hours,display_name,tarea
0,"[345, Planta1]",2025-10-15 19:03:14,"[80719, BOLSA - DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.]",2025-10-14 11:50:29,"[650, CLARA MARLEN OSPINA CASTRO]",sistema,"[2, Produccion]",112363.0,False,2025-10-15 14:03:14,6763.0,6763.0,"[13571, MO13301]",2025-10-14 06:50:30,2025-10-15 19:03:14,"[57, EMBALAJE]",3.84,"[369, BRAYAN NAIR MUÑOZ SALAS]",33343,1.88,CLARA MARLEN OSPINA CASTRO,Produccion
1,"[345, Planta1]",2025-10-15 12:27:26,"[80719, BOLSA - DA2570 FILTRO AIRE DONSSON - PERKINS, FIAT, CATERPILLAR.]",2025-10-14 11:50:48,"[603, INGRID CAROLINA GOMEZ DELPRADO ]",sistema,"[2, Produccion]",88596.0,False,2025-10-15 07:27:26,3101.0,3101.0,"[13571, MO13301]",2025-10-14 06:50:48,2025-10-15 12:27:26,"[57, EMBALAJE]",8.07,"[345, Planta1]",33344,0.86,INGRID CAROLINA GOMEZ DELPRADO,Produccion
2,"[345, Planta1]",2025-10-14 14:31:46,"[77136, REPUJADO - DA4124 FILTRO AIRE 2_ CATERPILLAR.]",2025-10-14 11:51:36,"[459, ERNESTO VALENCIA ORJUELA]",sistema,"[2, Produccion]",0.0,False,2025-10-14 09:31:46,9610.0,9610.0,"[13180, MO12914]",2025-10-14 06:51:41,2025-10-14 14:31:46,"[54, CORTE]",84478.49,"[345, Planta1]",33345,2.67,ERNESTO VALENCIA ORJUELA,Produccion
3,"[345, Planta1]",2025-10-14 21:11:51,False,2025-10-14 11:52:28,"[553, KATHERIN ANDREA GIL AGUILAR]",produccion,"[12, Auxiliar producción]",0.0,False,2025-10-14 16:11:54,0.0,0.0,False,2025-10-14 06:52:28,2025-10-14 21:11:51,False,149710.83,"[1, Administrador Administrador ]",33346,9.32,KATHERIN ANDREA GIL AGUILAR,Auxiliar producción
4,"[345, Planta1]",2025-10-14 21:11:51,False,2025-10-14 11:52:36,"[635, JUAN DAVID GUTIERREZ CORTES]",produccion,"[11, Molderia]",0.0,False,2025-10-14 16:11:54,0.0,0.0,False,2025-10-14 06:52:36,2025-10-14 21:11:51,False,139566.61,"[1, Administrador Administrador ]",33347,9.32,JUAN DAVID GUTIERREZ CORTES,Molderia


In [100]:
df_ot = df[df["tipo"]!="sistema"] #Se quitan las de op ya que no caen bien

In [101]:
df_ot = df_ot[["display_name","__last_update","create_date","tipo","is_pause","delay_hours","inicio","tarea"]]
df_ot.sample(5)

,display_name,__last_update,create_date,tipo,is_pause,delay_hours,inicio,tarea
275,DAVID ESTEBAN MALAGON CETINA,2025-10-16 13:28:55,2025-10-16 11:41:34,produccion,False,1.79,2025-10-16 06:41:34,Alistamiento MP
800,JUAN DAVID GUTIERREZ CORTES,2025-10-22 14:43:22,2025-10-22 11:46:35,produccion,False,2.95,2025-10-22 06:46:35,Molderia
495,MARIA PAULA PARRA PINILLA,2025-10-17 18:09:56,2025-10-17 15:59:07,produccion,False,2.18,2025-10-17 10:59:07,Reproceso
153,ZORAIDA CHAPARRO ALVAREZ,2025-10-15 13:17:22,2025-10-15 13:10:21,produccion,False,0.12,2025-10-15 08:10:21,Reproceso
277,KATHERIN ANDREA GIL AGUILAR,2025-10-16 21:12:33,2025-10-16 11:43:12,produccion,False,9.49,2025-10-16 06:43:12,Auxiliar producción


In [102]:
# SISTEMA = ORDENES DE PRODUCCION / PRODUCCION = OTRAS TAREAS / OTROS = COSAS QUE NO SON DE PRODUCCION

# TIEMPOS POR OP

In [124]:
import xmlrpc.client
from datetime import date, timedelta
import pandas as pd

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)
# ===============================
# 1. Conexión con Odoo
# ===============================

ODOO_URL = "https://donsson.com"
ODOO_DB = "Donsson_produccion"
ODOO_USERNAME = "juan.cano@donsson.com"
ODOO_PASSWORD = "1000285668"

# Autenticación
common = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/common")
uid = common.authenticate(ODOO_DB, ODOO_USERNAME, ODOO_PASSWORD, {})
if not uid:
    print("Error de autenticación. Verifica tus credenciales.")
    exit()
models = xmlrpc.client.ServerProxy(f"{ODOO_URL}/xmlrpc/2/object")

# ===============================
# 2. Definir rango de semanas
# ===============================

# Número de semanas que quieres descargar (puedes cambiarlo)
NUM_SEMANAS = weeks

# Fecha de hoy
hoy = date.today()
# Fecha de inicio según número de semanas atrás
fecha_inicio = hoy - timedelta(weeks=NUM_SEMANAS)

FECHA_INICIO = fecha_inicio.strftime("%Y-%m-%d")
FECHA_FIN = hoy.strftime("%Y-%m-%d")

print(f"📅 Descargando registros desde {FECHA_INICIO} hasta {FECHA_FIN}")

# ===============================
# 3. Leer ventas perdidas
# ===============================

ops_ids = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "search",
    [[["create_date", ">=", FECHA_INICIO], ["create_date", "<=", FECHA_FIN]]]
)

t_op = models.execute_kw(
    ODOO_DB, uid, ODOO_PASSWORD,
    "mrp.tiempos.trabajo", "read",
    [ops_ids],
    {"fields": []}
)



#"display_name","inicio","finalizo","is_pause"

df_op = pd.DataFrame(t_op)

df_op['Display_name'] = df_op['operario'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['OP'] = df_op['production_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))

df_op['centro_produccion'] = df_op['workcenter_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v))


df_op = df_op.drop(columns=["operario", "production_id", "workcenter_id","write_uid","id","display_name","create_uid","wc_line_id"])


📅 Descargando registros desde 2025-10-13 hasta 2025-10-27


In [125]:
df_op.sample(10)

,__last_update,horas_evento,create_date,fecha_evento,product_fabricado_id,fuerza_lab_id,evento,segundos_evento,write_date,Display_name,OP,centro_produccion
1503,2025-10-17 18:45:51,0.00,2025-10-17 18:45:51,2025-10-17 13:45:51,"[17390, [DAR03430025] DA3430 FILTRO AIRE MAZDA TURBO 4.5 (025 DA3430)]","[113667, PLISADORA - DA3430 FILTRO AIRE MAZDA TURBO 4.5]",iniciar,0.0,2025-10-17 18:45:51,DANILO PUPO SIERRA,MO13339,PLISADO
731,2025-10-17 18:16:34,1.29,2025-10-15 19:42:56,2025-10-15 14:42:56,"[16854, [DAR02139025] DA2139 FILTRO AIRE FORD (025 DA2139)]","[113387, CONTROL DE CALIDAD - DA2139 FILTRO AIRE FORD]",finalizar,4630.0,2025-10-17 18:16:34,YENNY KATERINE PATAQUIVA NEUSA,MO13260,EMBALAJE
2298,2025-10-21 21:12:07,1.55,2025-10-21 21:12:07,2025-10-21 16:12:07,"[17990, [DAE09048025] DA9048 FILTRO AIRE 2o. JAC 1035 (025 DA9048)]","[113886, TECNONEXT DURO - DA9048 FILTRO AIRE 2o. JAC 1035]",pausa,5573.0,2025-10-21 21:12:07,JUAN DAVID GUTIERREZ CORTES,MO13242,INYECCION
164,2025-10-14 15:59:18,0.00,2025-10-14 15:59:18,2025-10-14 10:59:18,"[17132, [DAB02702025] DA2702 FILTRO AIRE 1_ ISUZU,CHEV (FVR, FVZ), IHC, J.DEERE. (025 DA2702)]","[113215, EMBALAJE - DA2702 FILTRO AIRE 1_ ISUZU,CHEV (FVR, FVZ), IHC, J.DEERE.]",iniciar,0.0,2025-10-14 15:59:18,DEICY MILENA LOPEZ VELASQUEZ,MO13283,EMBALAJE
1273,2025-10-17 14:02:22,1.25,2025-10-17 14:02:22,2025-10-17 09:02:22,"[42181, [DAB08189025] DA8189 FILTRO AIRE HINO 700 (025 DA8189)]","[113571, MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA8189 FILTRO AIRE HINO 700 ]",pausa,4492.0,2025-10-17 14:02:22,ERIKA YULIEDT UMAÑA SABOGAL,MO13337,EMBALAJE
1487,2025-10-17 18:18:59,1.47,2025-10-17 18:18:59,2025-10-17 13:18:59,"[17284, [DAB02968025] DA2968 FILTRO AIRE- HINO (025 DA2968)]","[113654, MARCADO/HOTMELT/ARANDELA/EMPAQUE - DA2968 FILTRO AIRE- HINO ]",pausa,5308.0,2025-10-17 18:18:59,ERIKA YULIEDT UMAÑA SABOGAL,MO13315,EMBALAJE
367,2025-10-14 21:11:51,2.45,2025-10-14 21:11:51,2025-10-14 16:11:51,"[17634, [DAB04598025] DA4598 FILTRO AIRE- CATERPILLAR. (025 DA4598)]","[113289, ENTUBADORA 4 - DA4598 FILTRO AIRE- CATERPILLAR.]",pausa,8821.0,2025-10-14 21:11:51,JOSE DANIEL PUPO SIERRA,MO13309,CORTE
3592,2025-10-24 20:11:11,0.68,2025-10-24 20:11:11,2025-10-24 15:11:11,"[17296, [DAB02976025] DA2976 FILTRO AIRE MACK GRANITE (025 DA2976)]","[114245, POBLAN - DA2976 FILTRO AIRE MACK GRANITE]",pausa,2462.0,2025-10-24 20:11:11,ALVARO MELO PRADA,MO13374,CORTE
1945,2025-10-21 11:41:31,0.00,2025-10-21 11:41:31,2025-10-21 06:41:31,"[23573, [DAB08145025] DA8145 FILTRO AIRE EXCAVADORA RHINO REX210 (025 DA8145)]","[113792, ENTUBADORA 2 - DA8145 FILTRO AIRE EXCAVADORA RHINO REX210]",iniciar,0.0,2025-10-21 11:41:31,FRANCY CAROLINA PINZON CHACON,MO13349,CORTE
1573,2025-10-21 15:40:17,1.57,2025-10-18 15:03:06,2025-10-18 10:03:06,"[16926, [DAR02304025] DA2304 FILTRO AIRE 1_ CHR 7.2, NISSAN (025 DA2304)]","[113682, REPUJADO - DA2304 FILTRO AIRE 1_ CHR 7.2, NISSAN]",pausa,5634.0,2025-10-21 15:40:17,ERNESTO VALENCIA ORJUELA,MO13271,CORTE
